In [1]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train_df = TabularDataset("data/final_train.csv")
train_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:
label = "Exited"
models_path = "agModels-v1"

In [4]:
predictor = TabularPredictor(label=label, path=models_path).fit(train_df)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-v1"
=================== System Info ===================
AutoGluon Version:  1.0.1b20240101
Python Version:     3.8.18
Operating System

In [5]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2     0.8644    accuracy       0.176392  14.525572                0.002513           0.326056            2       True          7
1     RandomForestEntr     0.8604    accuracy       0.051568   6.529227                0.051568           6.529227            1       True          4
2     RandomForestGini     0.8588    accuracy       0.049553   5.679912                0.049553           5.679912            1       True          3
3       ExtraTreesGini     0.8560    accuracy       0.040522   1.930913                0.040522           1.930913            1       True          5
4       ExtraTreesEntr     0.8560    accuracy       0.051475   1.899081                0.051475           1.899081            1       True          6
5       KNeighborsUnif     0.7584    a

/Users/martinoywa/miniconda3/envs/2023_kaggle_playground_s4/lib/python3.8/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.7584,
  'KNeighborsDist': 0.7436,
  'RandomForestGini': 0.8588,
  'RandomForestEntr': 0.8604,
  'ExtraTreesGini': 0.856,
  'ExtraTreesEntr': 0.856,
  'WeightedEnsemble_L2': 0.8644},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': ['KNeighborsUnif'],
  'KNeighborsDist': ['KNeighborsDist'],
  'RandomForestGini': ['RandomForestGini'],
  'RandomForestEntr': ['RandomForestEntr'],
  'ExtraTreesGini': ['ExtraTreesGini'],
  'ExtraTreesEntr': ['ExtraTreesEntr'],
  'WeightedEnsemble_L2': ['WeightedEnsemble_L2']},
 'model_fit_times': {'KNeighborsUnif': 0.0594637393951416,
  'KNeighborsDist': 0.06475114822387695,
  'RandomForestGini': 5.679912090301514,
  'RandomForestEntr': 6.

In [6]:
test_df = TabularDataset("data/final_test.csv")
test_df.head()

Loaded data from: data/final_test.csv | Columns = 9 / 9 | Rows = 110023 -> 110023


,id,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,586,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,683,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,656,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,681,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,752,38.0,10,121263.62,1,1.0,0.0,139431.00


In [7]:
ids = test_df["id"]
drop_cols = ["id"]
train_df = test_df.drop(drop_cols, axis=1)

In [8]:
predictor = TabularPredictor.load(models_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict_proba(test_df)[1]
print("Predictions:  \n", y_pred[:10])

Predictions:  
 0    0.073599
1    0.718649
2    0.031067
3    0.227396
4    0.247053
5    0.018042
6    0.036639
7    0.104007
8    0.749093
9    0.016521
Name: 1, dtype: float64


In [9]:
# submission
submission_df = pd.DataFrame({"id": ids,
                             "Exited": y_pred})
submission_df.head()

,id,Exited
0,165034,0.073599
1,165035,0.718649
2,165036,0.031067
3,165037,0.227396
4,165038,0.247053


In [11]:
submission_df.to_csv("submissions/autogluon_WeightedEnsemble_L2-v1.csv", index=False)